# 🧰 March‑Test Snapshot Notebook 2.0  
This version adds  

* **Snapshot after *every gap*** → an extra snapshot recorded after a whole element finishes.  
* **SOLID‑ish structure** – each core task lives in its own function for single‑responsibility.  

Just edit **`INPUT_MARCH_TEXT`** and run _all_ cells.  


In [68]:
import re, copy, itertools
import pandas as pd

In [69]:
def parse_march(text:str):
    """Return list of (dir_tag, [ops]) tuples."""
    patt = re.compile(r"([abdf])\s*\{([^}]*)\}", re.I)
    elements=[]
    for m in patt.finditer(text):
        dir_tag=m.group(1).lower()
        ops=[tok for tok in re.split(r"\s+", m.group(2).strip()) if tok]
        elements.append((dir_tag, ops))
    return elements

In [70]:
# 測試parse_march
if __name__ == "__main__":
    march_text = "a {op1 op2} b {op3} d {op4 op5}"
    parsed = parse_march(march_text)
    print(parsed)  # Expected: [('a', ['op1', 'op2']), ('b', ['op3']), ('d', ['op4', 'op5'])]



[('a', ['op1', 'op2']), ('b', ['op3']), ('d', ['op4', 'op5'])]


In [71]:
def addr_order(dir_tag, n):
    return list(range(n)) if dir_tag in ('a','b') else list(range(n-1,-1,-1))

In [72]:
# 測試addr_order
if __name__ == "__main__":
    dir_tag = 'a'
    n = 3
    order = addr_order(dir_tag, n)
    print(f"Address order for '{dir_tag}': {order}")

    dir_tag = 'b'
    order = addr_order(dir_tag, n)
    print(f"Address order for '{dir_tag}': {order}")

    dir_tag = 'd'
    order = addr_order(dir_tag, n)
    print(f"Address order for '{dir_tag}': {order}")


Address order for 'a': [0, 1, 2]
Address order for 'b': [0, 1, 2]
Address order for 'd': [2, 1, 0]


In [73]:
def execute(op:str, D:list, C:list, addr:int):
    code,val=op[0].upper(),int(op[1])
    if code=='W':
        D[addr]=val
    elif code=='C':
        C[addr]=val
    # R does not change state
    return

In [74]:
# 測試execute
def print_state(Op, D, C):
    print(f"Operation: {Op}", end=': ')
    for i in range(len(D)):
        if i == len(D) - 1:
            print(f"{D[i]}{C[i]}", end='')
        else:
            print(f"{D[i]}{C[i]}", end=',')
    print()

if __name__ == "__main__":
    D = [0, 0, 0]
    C = [0, 0, 0]
    addr = 1
    execute('W1', D, C, addr)
    print_state('W1', D, C)

    execute('C1', D, C, addr)
    print_state('C1', D, C)

    execute('R0', D, C, addr)  # R does not change state
    print_state('R0', D, C)

Operation: W1: 00,10,00
Operation: C1: 00,11,00
Operation: R0: 00,11,00


In [75]:
def dc_state(D,C):
    return '{' + ', '.join(('X' if d is None else str(d))+('X' if c is None else str(c))
                           for d,c in zip(D,C)) + '}'

In [76]:
def combo(D,C,agg,vic):
    Da='X' if D[agg] is None else str(D[agg])
    Dv='X' if D[vic] is None else str(D[vic])
    Ia='X' if C[agg] is None else str(C[agg])
    Iv='X' if C[vic] is None else str(C[vic])
    return f'{{{Da},{Dv},{Ia},{Iv}}}'

In [77]:
# 測試 dc_state
if __name__ == "__main__":
    D = [0, 0, 0]
    C = [0, 0, 0]
    addr = 1
    execute('W1', D, C, addr)
    print("After W1:")
    state = dc_state(D, C)
    print(f"DC State: {state}")
    comb = combo(D, C, 0, 1)
    print(f"Combo State: {comb}")

    execute('C1', D, C, addr)
    print("After C1:")
    state = dc_state(D, C)
    print(f"DC State: {state}")
    comb = combo(D, C, 0, 1)
    print(f"Combo State: {comb}")

    execute('R0', D, C, addr)  # R does not change state
    print("After R0:")
    state = dc_state(D, C)
    print(f"DC State: {state}")
    comb = combo(D, C, 2, 1)
    print(f"Combo State: {comb}")

    execute('C0', D, C, addr)  # R does not change state
    print("After R0:")
    state = dc_state(D, C)
    print(f"DC State: {state}")
    comb = combo(D, C, 2, 1)
    print(f"Combo State: {comb}")

After W1:
DC State: {00, 10, 00}
Combo State: {0,1,0,0}
After C1:
DC State: {00, 11, 00}
Combo State: {0,1,0,1}
After R0:
DC State: {00, 11, 00}
Combo State: {0,1,0,1}
After R0:
DC State: {00, 10, 00}
Combo State: {0,1,0,0}


In [78]:
def simulate(elements, num_cells=3, victim=1):
    D=[None]*num_cells; C=[None]*num_cells
    snapshots={}          # idx -> {DC,DC,DC}
    combo_sources={}       # combo -> [ '(idx,addrX)' ]
    op_labels=[]           # per element: list of (op, idx_before) plus idx_after

    idx=1  # snapshot index

    for dir_tag, ops in elements:
        per_elem=[]
        # Traverse addresses
        for addr in addr_order(dir_tag,num_cells):
            for op in ops:
                execute(op,D,C,addr)
                # record snapshot after each op
                if addr==victim:
                    snapshots[idx]=dc_state(D,C)
                    for agg in (0,num_cells-1):
                        c=combo(D,C,agg,victim)
                        combo_sources.setdefault(c,[]).append(f'({idx}, addr{agg})')
                    # save label only for victim‑address ops
                    per_elem.append((op,idx))
                    idx+=1
        # extra snapshot after element finishes
        snapshots[idx]=dc_state(D,C)
        for agg in (0,num_cells-1):
            c=combo(D,C,agg,victim)
            combo_sources.setdefault(c,[]).append(f'({idx}, addr{agg})')
        per_elem.append(('END',idx))   # marker for final
        idx+=1
        op_labels.append(per_elem)
    return snapshots, combo_sources, op_labels

In [79]:
def render_expanded(elements, op_labels):
    lines=[]
    for (dir_tag,ops), labels in zip(elements,op_labels):
        # labels aligned with ops plus final 'END'
        label_iter=iter(labels)
        rendered=[]
        for op in ops:
            lab_op=next(label_iter)
            rendered.append(f'({lab_op[1]}) {op}')
        # final END label
        end_lab=next(label_iter)
        rendered.append(f'({end_lab[1]})')
        lines.append(f"{dir_tag} {{ " + ' '.join(rendered) + " }}")
    return lines

In [80]:
#測試 render_expanded
if __name__ == "__main__":
    elements = parse_march("a {W0 C1} b {R2} d {C0 W1}")
    snapshots, combos, op_labels = simulate(elements)
    expanded_lines = render_expanded(elements, op_labels)

    print('# Expanded March test with indexes\n')
    for l in expanded_lines:
        print(l)

# Expanded March test with indexes

a { (1) W0 (2) C1 (3) }}
b { (4) R2 (5) }}
d { (6) C0 (7) W1 (8) }}


In [81]:
def make_combo_df(combo_sources):
    rows=[(c,', '.join(srcs)) for c,srcs in combo_sources.items()]
    df=pd.DataFrame(rows,columns=['{Da,Dv,Ia,Iv}','Sources']).sort_values('{Da,Dv,Ia,Iv}').reset_index(drop=True)
    df.index += 1  # 將索引從 1 開始
    return df

In [96]:
# --------- USER INPUT ---------
# INPUT_MARCH_TEXT = """b { W0  C0 } a { R0  W1  C1 } a { R1  W0  C0 } d { R0  W1  C1 } d { R1  W0  C0 } b { R0 }"""
INPUT_MARCH_TEXT = """b { W0  C00 } a { R0  C11  W1 } a { R1  C01  W0 } d { R0  C11  W1 } d { R1  C01  W0 } b { R0 }"""
# INPUT_MARCH_TEXT = """b { W0  C0 } a { R0  W1  C1 } a { R1  W0  C0 } d { R0  W1  C1 } d { R1  W0  C0 } b { R0  C1 }"""
# INPUT_MARCH_TEXT = """b { W0  C1 } a { R0  W1  C0 } a { R1  W0  C1 } d { R0  W1  C0 } d { R1  W0  C1 } b { R0  C1 }"""
# INPUT_MARCH_TEXT = """b { W0  C0 } a { R0  C1  W1 } a { R1  C0  W0 } d { R0  C1  W1 } d { R1  C0  W0 } b { R0  C1 }"""


elements=parse_march(INPUT_MARCH_TEXT)
snapshots, combos, op_labels = simulate(elements)
expanded_lines = render_expanded(elements, op_labels)

print('# Expanded March test with indexes\n')
for l in expanded_lines:
    print(l)

print('\n# {DC,DC,DC} snapshot list\n')
for i in range(1, max(snapshots.keys())+1):
    print(f'({i})  {snapshots[i]}')

# Expanded March test with indexes

b { (1) W0 (2) C00 (3) }}
a { (4) R0 (5) C11 (6) W1 (7) }}
a { (8) R1 (9) C01 (10) W0 (11) }}
d { (12) R0 (13) C11 (14) W1 (15) }}
d { (16) R1 (17) C01 (18) W0 (19) }}
b { (20) R0 (21) }}

# {DC,DC,DC} snapshot list

(1)  {00, 0X, XX}
(2)  {00, 00, XX}
(3)  {00, 00, 00}
(4)  {11, 00, 00}
(5)  {11, 01, 00}
(6)  {11, 11, 00}
(7)  {11, 11, 11}
(8)  {00, 11, 11}
(9)  {00, 10, 11}
(10)  {00, 00, 11}
(11)  {00, 00, 00}
(12)  {00, 00, 11}
(13)  {00, 01, 11}
(14)  {00, 11, 11}
(15)  {11, 11, 11}
(16)  {11, 11, 00}
(17)  {11, 10, 00}
(18)  {11, 00, 00}
(19)  {00, 00, 00}
(20)  {00, 00, 00}
(21)  {00, 00, 00}


In [95]:
print('# Expanded March test with indexes\n')
for l in expanded_lines:
    print(l)

print('\n# {Da,Dv,Ia,Iv} combinations\n')
df=make_combo_df(combos)
display(df)

# Expanded March test with indexes

b { (1) W0 (2) C0 (3) }}
a { (4) R0 (5) W1 (6) C1 (7) }}
a { (8) R1 (9) W0 (10) C0 (11) }}
d { (12) R0 (13) W1 (14) C1 (15) }}
d { (16) R1 (17) W0 (18) C0 (19) }}
b { (20) R0 (21) }}

# {Da,Dv,Ia,Iv} combinations



,"{Da,Dv,Ia,Iv}",Sources
1,"{0,0,0,0}","(2, addr0), (3, addr0), (3, addr2), (4, addr2)..."
2,"{0,0,0,1}","(9, addr0), (17, addr2)"
3,"{0,0,0,X}","(1, addr0)"
4,"{0,1,0,0}","(5, addr2), (13, addr0)"
5,"{0,1,0,1}","(6, addr2), (8, addr0), (14, addr0), (16, addr2)"
6,"{1,0,1,0}","(4, addr0), (10, addr2), (12, addr2), (18, addr0)"
7,"{1,0,1,1}","(9, addr2), (17, addr0)"
8,"{1,1,1,0}","(5, addr0), (13, addr2)"
9,"{1,1,1,1}","(6, addr0), (7, addr0), (7, addr2), (8, addr2)..."
10,"{X,0,X,0}","(2, addr2)"
